<a href="https://colab.research.google.com/github/Algon-33/TWI-Generation/blob/main/TWI%20GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#https://huggingface.co/docs/transformers/v4.23.1/en/training#finetune-a-pretrained-model

#Tests that the transformer library was installed correctly
from transformers import pipeline;
print(pipeline('sentiment-analysis')('we love you'))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998704195022583}]


In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
import random
%matplotlib inline

In [7]:
from transformers import GPT2Model, GPT2Config

# Initializing a GPT2 configuration

configuration = GPT2Config()

# Initializing a model from the configuration

model = GPT2Model(configuration)

# Accessing the model configuration

configuration = model.config

In [8]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer("Hello world")['input_ids']

[15496, 995]

In [9]:
!wget https://raw.githubusercontent.com/Algon-33/TWI-Generation/main/dataset.txt

--2022-10-26 23:40:57--  https://raw.githubusercontent.com/Algon-33/TWI-Generation/main/dataset.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104606 (102K) [text/plain]
Saving to: ‘dataset.txt.1’

dataset.txt.1       100%[===================>] 102.15K  --.-KB/s    in 0.003s  

2022-10-26 23:40:57 (32.7 MB/s) - ‘dataset.txt.1’ saved [104606/104606]



In [10]:
from re import S
# read in all the words
random.seed(925)
dataset = open('dataset.txt', 'r').read().splitlines()
dataset = list(map( lambda x: x.strip("]").strip("["), dataset))
random.shuffle(dataset)
dataset[0:8]

['cafe manager',
 'ambush commander',
 'reckless charge',
 'riders',
 'renowned tincturist',
 'heavyweight punch',
 'arbiters',
 'fashionistas']

In [11]:
tokenizer.pad_token = tokenizer.eos_token 

In [12]:
#This is wrong. You need to use the Dataset datastructure in order to use the tutorial you've been going through.
#https://huggingface.co/datasets/yelp_review_full#dataset-structure


tokens = tokenizer(dataset, padding=True, return_tensors="pt")

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [14]:
import numpy as np

import evaluate

metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [18]:
trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=dataset,

    eval_dataset=tokens,

    compute_metrics=compute_metrics,

)

In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6015
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2256


TypeError: ignored

In [20]:
from datasets import load_dataset

In [21]:
def tokenize_function(examples):

    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [24]:
D = load_dataset("yelp_review_full")

D["train"][100]
tokenized_datasets = D.map(tokenize_function, batched=True)

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [29]:
"""DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})"""

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [30]:
"""Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 650000
})"""#

tokenized_datasets["train"]

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 650000
})

In [1]:
#Dataset columns are ['label', 'text', 'input_ids', 'attention_mask']
#label is just an integer
#text is just a string of text
#input_ids is just the tokens
#mask is just what you'd expectf
tokenized_datasets["train"]["input_ids"]

NameError: ignored

In [28]:
tokenized_datasets["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 